In [51]:
import json 
import pandas as pd 
import numpy as np
with open('response1.json') as f:
    d = json.load(f)

# Responese2.json tiene valores raros de latitud en algunos casos, revisar antes de sumar al CSV

In [52]:
d["results"][0]["attributes"] 
#ahora veo una lista de atributos, cada uno de ellos es un diccionario con las mismas "keys" 

[{'id': 'ITEM_CONDITION',
  'value_id': '2230581',
  'value_name': 'Usado',
  'attribute_group_id': '',
  'source': 1,
  'name': 'Condición del ítem',
  'value_struct': None,
  'values': [{'id': '2230581', 'name': 'Usado', 'struct': None, 'source': 1}],
  'attribute_group_name': ''},
 {'id': 'HAS_TELEPHONE_LINE',
  'name': 'Línea telefónica',
  'attribute_group_id': 'CARACTERISTICAS',
  'values': [{'id': '242084',
    'name': 'No',
    'struct': None,
    'source': 4709975701260268}],
  'attribute_group_name': 'Características adicionales',
  'source': 4709975701260268,
  'value_id': '242084',
  'value_name': 'No',
  'value_struct': None},
 {'value_name': 'No',
  'value_struct': None,
  'attribute_group_id': 'COMOYAMEN',
  'attribute_group_name': 'Comodidades y amenities',
  'source': 4709975701260268,
  'id': 'HAS_AIR_CONDITIONING',
  'name': 'Aire acondicionado',
  'value_id': '242084',
  'values': [{'name': 'No',
    'struct': None,
    'source': 4709975701260268,
    'id': '242084'

In [53]:
#puedo iterar sobre la lista de atributos porque comparten las keys para hacer un nuevo diccionario
dict_global = {}
for j in range(len(d["results"])):
     dict_attr = {}
     for i in range(len(d["results"][j]["attributes"])): #numero de atributos por ítem, no siempre es 10
          dict_attr [d["results"][j]["attributes"][i]["name"]] = d["results"][j]["attributes"][i]["value_name"] 
     dict_global[d["results"][j]["id"]] = dict_attr

In [54]:
df_attr = pd.DataFrame.from_dict(dict_global)

In [55]:
df_att=df_attr.T.drop(['Tour virtual'], axis=1)

Los rasgos básicos de la base de datos son los que vienen desde el objeto de atributos del JSON

| Nombre | Tipo de variable |
| --- | --- |
| Condición del ítem | Binaria |
| Línea telefónica | Binaria |
| Dormitorios | Binaria |
| Superfice cubierta | Entero? m2|
| Baños | Entero |
| Ambientes | Entero | 
| Superficie total | Entero? m2| 
| Operación | Binario |
| Inmueble | Binario |

Deberia poder sacar las unidades de las columnas de area """.str.replace"""

In [56]:
df_att[["Superficie cubierta","Superficie total"]] = df_att[["Superficie cubierta","Superficie total"]].replace({'m²':''}, regex=True)

Extraigo otros valores que me pueden interesar

In [57]:
dict_select = {}
dict_global2 = {}

In [58]:
for i in range (len(d["results"])):
    if d["results"][i]["location"]["city"]["name"] == "Capital Federal":
        dict_select = {}
        dict_select["price"] = d["results"][i]["price"]
        dict_select["currency_id"] = d["results"][i]["currency_id"]
        dict_select["stop_time"] = d["results"][i]["stop_time"]
        dict_select["official_store_id"] = d["results"][i]["official_store_id"]
        dict_select["seller.id"] = d["results"][i]["seller"]["id"]
        dict_select["location.longitude"] = d["results"][i]["location"]["longitude"]
        dict_select["location.latitude"] = d["results"][i]["location"]["latitude"]
        dict_select["location.address_line"] = d["results"][i]["location"]["address_line"]
        dict_select["prices_last_updated"] = d["results"][i]["prices"]["prices"][0]["last_updated"]
        dict_global2[d["results"][i]["id"]] = dict_select

In [59]:
df_select = pd.DataFrame.from_dict(dict_global2)

In [60]:
df_select.T.head(2)

,price,currency_id,stop_time,official_store_id,seller.id,location.longitude,location.latitude,location.address_line,prices_last_updated
MLA1130578098,43500,ARS,2022-06-03T04:00:00.000Z,2695,155651025,-58.457867,-34.61696,Donato Alvarez al 700,2022-04-04T18:23:46Z
MLA1130495400,60000,ARS,2022-06-02T15:59:33.000Z,None,173700530,-58.461285,-34.54327,"Av. del Libertador & Manzanares, Buenos Aires,...",2022-04-03T02:49:46Z


In [61]:
df_select_ars= df_select.T[df_select.T.currency_id == "ARS"]

Había 8 en dolares solamente. Podria tomarme la molestia de seleccionar solo los deptos que esten en dolares, en vez de ajustar el precio dependiendo de la fecha. Podria haces Lotes bimestrales con los precios promedio del dolar 

In [62]:
df_features = df_select.T[["location.longitude","location.latitude","price"]].join(df_att)  

Cambio el nombre y corrijo los tipos de cada variable

In [65]:
df_features.rename(columns = {'location.longitude':'longitud', 'location.latitude':'latitud', 'price':'precio'}, inplace = True)
df_features = df_features.astype({'precio' : "int64", 'longitud': 'float' , "latitud" : "float" , "Dormitorios" : "int32" , "Superficie cubierta" : "float", "Baños" : "int32" , "Ambientes" : "int32" , "Superficie total" : "float" })
df_features.dtypes

longitud               float64
latitud                float64
precio                   int64
Condición del ítem      object
Línea telefónica        object
Aire acondicionado      object
Dormitorios              int32
Superficie cubierta    float64
Baños                    int32
Ambientes                int32
Superficie total       float64
Inmueble                object
dtype: object

Reviso que todas las propiedades sean alquileres

In [66]:
df_features["Operación"].str.contains("Alquiler").all()

KeyError: 'Operación'

Depues elimino esa columna para que no sea redundante y exporto la tabla para poder seguir con un EDA en otro notebook. Para seguir agregando otros datos a la tabla se tiene que aplicar el modo append de to_csv con el parametro header = false para que no se agregue el nombre de las columnas como renglon

In [72]:
df_features.drop("Operación",axis=1,inplace=True)
df_features.to_csv("tabla de propiedades.csv")
#df.to_csv('my_csv.csv', mode='a', header=False)

KeyError: "['Operación'] not found in axis"

In [67]:
import folium as fl

In [68]:
fg = fl.FeatureGroup(name="My Map")


Se podria hacer un intento naive, solo usar el metro cuadrado con OLS. Despues sumar la localización y pasar a MLS. 
Se podra usar una sola medida espacial? distancia de una arteria urbana? Los precios podrian indicar una estructura oculta en la geografia de la ciudad?

In [69]:
for i in range(len(df_features)):
    fg.add_child(fl.Marker(location=[df_features["latitud"][i],df_features["longitud"][i]], popup=df_features["precio"][i]))

In [70]:
map = fl.Map(location=[-34.60165478206169, -58.38082359962473])

In [71]:
map.add_child(fg)

In [ ]:
map.save("propiedades.html")

Todas las propiedades quedan en CABA porque así lo definí en el loop for que creo el diccionario.